# Badlands vtk stratigraphic mesh creation

If the stratigraphic structure is turned on in the XmL input file, **Badlands** produces sedimentary layers Hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we build a vtk **structured grid** based on the stratal layers that could be visualise directly in **Paraview/Visit**.

In [ ]:
%matplotlib inline

import numpy as np

# Import badlands grid generation toolbox
import badlands_companion.stratalMesh as mesh

# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

# Loading stratigraphic file

First we need to load one of the stratigraphic file. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.hdf5`

with T the display time index and X the number of the partition (used in the parallel version). In cases where you ran your simulation in parallel you will also need to give the number of CPUs used (_cpus_).

To load a file you will need to give the folder name and the number of processors used in your simulation.

For more information regarding the function uncomment the following line.

In [ ]:
folder = 'outbdls'
vtkMesh = mesh.stratalMesh(folder+'/h5')

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
vtkMesh.loadStratigraphy(timestep=30) #last timestep -1

# Building the vtk unstructured grid

We then build the stratigraphic mesh for the considered time interval. 

You need to specify the folder where you want your output to be stored.

The mesh file names will have the following convention:

- `stratalMesh.time`T`.vts`

with T the display time index.

In [ ]:
vtkMesh.buildMesh(outfolder='outbdls')

In [ ]:
folder = 'outbdls'
vtkMesh = mesh.stratalMesh(folder+'/h5')

In [ ]:
import glob,re
folder2 = 'outbdls'
stepCounter = len(glob.glob1(folder2+"/xmf/","tin.time*"))-1
print("Number of visualisation steps created: ",stepCounter)

In [ ]:
pvd_file = folder+'/stratiView.pvd'


In [ ]:
f= open(str(pvd_file),'w')

f.write('<?xml version="1.0"?>\n')
f.write('<VTKFile type="Collection" version="0.1"\n')
f.write('         byte_order="LittleEndian"\n')
f.write('         compressor="vtkZLibDataCompressor">\n')
f.write('  <Collection>\n')

kk = 0
for k in range(1,stepCounter+1,1):#change timestep 10/50/,...
    vtkMesh.loadStratigraphy(timestep=k)
    vtkMesh.buildMesh(outfolder=folder+'/h5')
    # Get the sea level history and depositional time for each stratigraphic layer
    with open(folder+'/xmf/tin.time'+str(k)+'.xmf') as fp:
        line = fp.readline()
        while line:
            if 'Time' in line:
                val = [float(s) for s in re.findall(r'-?\d+\.?\d*', line)]
                time = val[0]
            line = fp.readline()
    print('Rendering for time step '+str(k)+': '+str(time)+' years')
    f.write('    <DataSet timestep="%s" group="" part="%d" file="xmf/strat.time%d.vtm"/>\n'%(time,kk,kk))
    # Create the VTM
    vtm_file = folder+'/xmf/strat.time'+str(kk)+'.vtm'
    fvtm= open(str(vtm_file),'w')
    fvtm.write('<?xml version="1.0"?>\n')
    fvtm.write('<VTKFile type="vtkMultiBlockDataSet" version="1.0">\n')
    fvtm.write(' <vtkMultiBlockDataSet>\n')
    fvtm.write('  <DataSet index="0" name="" file="../h5/stratalMesh.time%d.vts"/>  \n'%k)
    fvtm.write(' </vtkMultiBlockDataSet>\n')
    fvtm.write('</VTKFile>\n')
    fvtm.close()
    kk += 1



f.write('  </Collection>\n')
f.write('</VTKFile>\n')
f.close()